In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math

In [0]:
def imanticorrect(img, L):
    canvas = img.astype('float')
    canvas = canvas * L
    return canvas

# Error Metric

# Single light
def sl(Le, Lt):
  Le = Le.astype(np.float)
  Lt = Lt.astype(np.float)
  s = (np.linalg.norm(Le)*np.linalg.norm(Lt))
  if s == 0:
    return math.pi/2
  return math.acos((np.dot(Le,Lt)/s).clip(-1,1))

# Multiple Light
def ml(Le, Lt):
  Le = Le.astype(np.float)
  Lt = Lt.astype(np.float)
  denominator = np.linalg.norm(Le, axis=2) * np.linalg.norm(Lt, axis=2)
  numerator = (Le * Lt).sum(axis=2)
  epsilons = np.zeros((denominator != 0).sum())
  epsilons = numerator[denominator != 0] / denominator[denominator != 0]
  epsilons = np.arccos(epsilons.clip(-1,1))
  epsilons[np.isnan(epsilons)] = math.pi/2 #
  average = epsilons.mean()
  return average

# Generator of Multiple Light

def lightGenerate(imgshape, k, sigma=-1):
  if sigma == -1:
    sigma = math.ceil(min(imgshape)/64)
    sigma = max(abs(math.floor(np.random.random()*4 - 2 + sigma + 0.5)), 1)
  height = imgshape[0]
  width = imgshape[1]
  x = np.arange(1, width + 1)
  y = np.arange(1, height + 1)
  cor = np.meshgrid(x, y)
  cor_data = np.stack(cor, axis=2).reshape(-1,2).astype(np.float32)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  flags = cv2.KMEANS_RANDOM_CENTERS
  compactness,labels,centers = cv2.kmeans(cor_data, k, None, criteria, 1, flags)
  canvas = np.zeros((height, width, 3), np.uint8)
  c_labels = labels.reshape((height, width))
  for i in range(k):
    canvas[:,:,0][c_labels == i] = np.random.randint(0,256)
    canvas[:,:,1][c_labels == i] = np.random.randint(0,256)
    canvas[:,:,2][c_labels == i] = np.random.randint(0,256)
  return cv2.GaussianBlur(canvas,(math.ceil(6*sigma+1),math.ceil(6*sigma+1)),sigma).astype(np.float)

In [0]:
def errorMeasure(method, kw, imgs, lights=None, num=1000, err_metric_method=ml):
    if lights == None:
        num_imgs = len(imgs)
        set_imgs = []
        set_lights = []
        set_errors = []
        for i in range(num):
            n = math.floor(np.random.random() * num_imgs)
            set_lights.append(lightGenerate(imgs[n].shape[:-1], 4 + math.floor(np.random.random() * 4)))
            set_imgs.append(imanticorrect(imgs[n], set_lights[-1]))
    else:
        set_imgs = imgs
        set_lights = lights
    for i in range(len(set_imgs)):
        set_errors.append(err_metric_method(method(set_imgs[i], **kw), set_lights[i]))
    return np.mean(set_errors), np.median(set_errors)

In [0]:
import sys
import os
sys.path.append('/content/drive/My Drive/Color Constancy/Source-code/code/')

In [0]:
from DOCC import DOCC
import BU
#...

#Ls = []
# imgs = map(lambda u:cv2.resize(cv2.imread('/content/drive/My Drive/Color Constancy/Resource/ColorChecker Dataset/Corrected/'+u)[:,:,::-1], (0, 0), fx=0.1, fy=0.1), 
#            os.listdir('/content/drive/My Drive/Color Constancy/Resource/ColorChecker Dataset/Corrected'))
imgs = map(lambda u:cv2.imread('/content/drive/My Drive/Color Constancy/Resource/ColorChecker Dataset/Corrected/'+u)[:,:,::-1], 
            os.listdir('/content/drive/My Drive/Color Constancy/Resource/ColorChecker Dataset/Corrected'))

In [0]:
imgs = list(imgs)

In [0]:
def doNothing(img):
    return img
#     return np.ones(img.shape)

def BU_method1(img, R1_sigma, R2_sigma, k):
    img_BU = BU.BU(img, R1_sigma, R2_sigma, k)
    return img_BU.method1()[0]
  
def BU_method2(img, R1_sigma, R2_sigma, k):
    img_BU = BU.BU(img, R1_sigma, R2_sigma, k)
    return img_BU.method2()[0]

In [0]:
# DOCC 
err = errorMeasure(DOCC, {'R1_sigma':1, 'R2_sigma':3, 'k':0.6, 'pooling':'none'}, imgs, num=600)
err[0]/math.pi*180, err[1]/math.pi*180

In [0]:
# BU method1
err = errorMeasure(BU_method1, {'R1_sigma':1, 'R2_sigma':2, k:0.3}, imgs, num=600)
err[0]/math.pi*180, err[1]/math.pi*180

In [0]:
# BU method2
err = errorMeasure(BU_method2, {'R1_sigma':1, 'R2_sigma':2, k:0.3}, imgs, num=600)
err[0]/math.pi*180, err[1]/math.pi*180

In [0]:
# Do nothing
imgs = [cv2.imread('/content/drive/My Drive/Color Constancy/Resource/ColorChecker Dataset/Corrected/54.png')[:,:,::-1]]
err = errorMeasure(doNothing, {}, imgs, num=50)
err[0]/math.pi*180, err[1]/math.pi*180

(4.53331325366111, 4.473571388340897)